## Naive Bayes

### Training des Klassifikators

Klassenprioris:

- $P(O) = \frac{4}{7}$
- $P(M) = \frac{3}{7}$

Bedingte Wahrscheinlichkeiten:

**Alter**
- $P(<35 \mid O) = \frac{2}{4}$
- $P(<35 \mid M) = \frac{1}{3}$

**Einkommen**
- $P(niedrig \mid O) = \frac{1}{4}$
- $P(niedrig \mid M) = \frac{2}{3}$

**Bildung**
- $P(Bachelor \mid O) = \frac{1}{4}$
- $P(Bachelor \mid M) = \frac{1}{3}$

### Klassifikation eines Wählers

Wähler $(<35,\ niedrig,\ Bachelor)$.

$$
P(K \mid x) \propto P(K)\cdot P(<35 \mid K)\cdot P(niedrig \mid K)\cdot P(Bachelor \mid K)
$$

- $P(O \mid x) = \frac{1}{56}$
- $P(M \mid x) = \frac{2}{63}$

Da $P(M \mid x) > P(O \mid x)$, wird der Wähler Kandidat **M** zugeordnet.

### Arbeitsweise des Klassifikators

Naive Bayes berechnet für jede Klasse die a-posteriori-Wahrscheinlichkeit auf Basis der Trainingsdaten.  
Dabei wird angenommen, dass die Merkmale bedingt unabhängig voneinander sind.  
Die Klasse mit der höchsten berechneten Wahrscheinlichkeit wird ausgewählt.

# Spam Mail Klassifikation mit Naive Bayes

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Datensatz laden
df = pd.read_csv("spam_ham_dataset.csv")

# Bag-of-Words-Vektorisierung
vectorizer = CountVectorizer(
    lowercase=True,
    stop_words="english",
    max_features=5000
)

X = vectorizer.fit_transform(df["text"])
y = df["label_num"]

print("Anzahl Dokumente:", X.shape[0])
print("Anzahl Merkmale:", X.shape[1])

Anzahl Dokumente: 5171
Anzahl Merkmale: 5000


## Aufgabe 2: Naive Bayes Klassifikator

Für die Klassifikation wird ein **Multinomial Naive Bayes Klassifikator** eingesetzt. Dieses Modell eignet sich besonders für Textdaten, da es auf diskreten Merkmalen wie Wortzählungen basiert.

Der Klassifikator schätzt:

- die Klassenprioren $P(c)$ für $ham$ und $spam$
- die bedingten Wahrscheinlichkeiten $P(w_i \mid c)$ für jedes Wort $w_i$ gegeben eine Klasse $c$

Die Klassenzuordnung eines Dokuments erfolgt über die Maximierung der logarithmierten Posterior-Wahrscheinlichkeit:

$$
\hat{c}
=
\arg\max_{c}
\left(
\log P(c)
+
\sum_i \log P(w_i \mid c)
\right)
$$

Die Implementierung erfolgt mithilfe der Bibliothek **scikit-learn**, die eine effiziente und erprobte Umsetzung bereitstellt.


In [6]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

nb.fit(X_train, y_train)

print("Gelernte Klassenprioren:", nb.class_log_prior_)

y_pred = nb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=["ham", "spam"])

print("Accuracy:", accuracy)
print(report)

Gelernte Klassenprioren: [-0.34234048 -1.23824126]
Accuracy: 0.9565217391304348
              precision    recall  f1-score   support

         ham       0.98      0.95      0.97       735
        spam       0.89      0.96      0.93       300

    accuracy                           0.96      1035
   macro avg       0.94      0.96      0.95      1035
weighted avg       0.96      0.96      0.96      1035



Die Genauigkeit des Klassifikators liegt bei etwa $95.7\%$ und zeigt eine zuverlässige Trennung zwischen $spam$ und $ham$.

Zur Interpretation des Modells werden die wahrscheinlichsten Wörter pro Klasse betrachtet.

In [7]:
import numpy as np

feature_names = vectorizer.get_feature_names_out()

spam_top = np.argsort(nb.feature_log_prob_[1])[-10:]
ham_top = np.argsort(nb.feature_log_prob_[0])[-10:]

spam_words = [feature_names[i] for i in spam_top]
ham_words = [feature_names[i] for i in ham_top]

print("Typische Spam-Wörter:", spam_words)
print("Typische Ham-Wörter:", ham_words)

Typische Spam-Wörter: ['new', 'www', 'price', 'statements', 'information', '00', 'company', 'http', 'com', 'subject']
Typische Ham-Wörter: ['pm', 'cc', 'meter', 'deal', 'gas', '2000', 'subject', 'enron', 'hou', 'ect']


Für die Klasse $spam$ dominieren Begriffe aus Werbung, Preisen und Webadressen, während $ham$ vor allem durch geschäftliche und unternehmensinterne Begriffe charakterisiert ist. Trotz der vereinfachenden Unabhängigkeitsannahme liefert der Naive Bayes Klassifikator sehr gute Ergebnisse für die Spam-Erkennung.